In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
from send_posture_status import send_posture_status

last_status = None
last_send_time = time.time()  

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)

is_calibrated = False
calibration_frames = 0
calibration_shoulder_angles = []
calibration_neck_angles = []
calibration_head_distances = []
calibration_left_shoulder_heights = []
calibration_right_shoulder_heights = []
calibration_nose_distances = []
CALIBRATION_FRAME_COUNT = 100

def calculate_angle(point1, point2, point3):
    a = np.array(point1)
    b = np.array(point2)
    c = np.array(point3)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360.0 - angle

    return angle

def calculate_distance_y(point1, point2):
    return np.abs(point1[1] - point2[1])

def draw_angle(frame, point1, point2, point3, angle, color):
    cv2.putText(frame, str(int(angle)), point2, cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

def draw_shapes(frame, left_shoulder, right_shoulder, left_ear, right_ear):
    midpoint = ((left_shoulder[0] + right_shoulder[0]) // 2, (left_shoulder[1] + right_shoulder[1]) // 2)

    cv2.line(frame, left_shoulder, midpoint, (255, 255, 255), 2)
    cv2.line(frame, right_shoulder, midpoint, (255, 255, 255), 2)
    cv2.line(frame, left_shoulder, left_ear, (255, 255, 255), 2)
    cv2.line(frame, right_shoulder, right_ear, (255, 255, 255), 2)

    upper_midpoint = (midpoint[0], midpoint[1] - 100)
    cv2.line(frame, midpoint, upper_midpoint, (255, 255, 255), 2)
    cv2.line(frame, left_ear, upper_midpoint, (255, 255, 255), 2)
    cv2.line(frame, right_ear, upper_midpoint, (255, 255, 255), 2)

    cv2.circle(frame, left_shoulder, 5, (255, 255, 255), -1)
    cv2.circle(frame, right_shoulder, 5, (255, 255, 255), -1)
    cv2.circle(frame, midpoint, 5, (255, 255, 255), -1)
    cv2.circle(frame, upper_midpoint, 5, (255, 255, 255), -1)

if cap.isOpened():
    time.sleep(5)
 
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        left_shoulder = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * frame.shape[1]),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * frame.shape[0]))
        right_shoulder = (int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * frame.shape[1]),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * frame.shape[0]))
        left_ear = (int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x * frame.shape[1]),
                    int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y * frame.shape[0]))
        right_ear = (int(landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x * frame.shape[1]), 
                     int(landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y * frame.shape[0]))
        
        nose = (int(landmarks[mp_pose.PoseLandmark.NOSE.value].x * frame.shape[1]),
                int(landmarks[mp_pose.PoseLandmark.NOSE.value].y * frame.shape[0]))

        chest = ((left_shoulder[0] + right_shoulder[0]) // 2, left_shoulder[1] + 50)

        shoulder_angle = calculate_angle(left_shoulder, right_shoulder, (right_shoulder[0], 0))
        neck_angle = calculate_angle(left_ear, left_shoulder, (left_shoulder[0], 0))
        nose_to_chest_distance = calculate_distance_y(nose, chest)

        shoulder_imbalance = calculate_distance_y(left_shoulder, right_shoulder)
        head_tilt_angle = calculate_angle(left_ear, left_shoulder, right_shoulder)
        
        if not is_calibrated and calibration_frames < CALIBRATION_FRAME_COUNT:
            calibration_shoulder_angles.append(shoulder_angle)
            calibration_neck_angles.append(neck_angle)
            calibration_head_distances.append(nose_to_chest_distance)
            calibration_left_shoulder_heights.append(left_shoulder[1])
            calibration_right_shoulder_heights.append(right_shoulder[1])
            calibration_frames += 1
            cv2.putText(frame, f"Kalibrasyon yapılıyor... {calibration_frames}/{CALIBRATION_FRAME_COUNT}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        elif not is_calibrated:
            shoulder_threshold = np.mean(calibration_shoulder_angles) - 1.5 * np.std(calibration_shoulder_angles)
            neck_threshold = np.mean(calibration_neck_angles) - 1.5 * np.std(calibration_neck_angles)
            nose_distance_threshold = np.mean(calibration_head_distances) - 1.5 * np.std(calibration_head_distances)
            shoulder_imbalance_threshold = np.abs(np.mean(calibration_left_shoulder_heights) - np.mean(calibration_right_shoulder_heights)) + 10
            head_tilt_angle_threshold = 30  

            is_calibrated = True
            print("Kalibrasyon tamamlandı.")

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        draw_shapes(frame, left_shoulder, right_shoulder, left_ear, right_ear)

        midpoint = ((left_shoulder[0] + right_shoulder[0]) // 2, (left_shoulder[1] + right_shoulder[1]) // 2)
        draw_angle(frame, left_shoulder, midpoint, (midpoint[0], 0), shoulder_angle, (255, 0, 0))
        draw_angle(frame, left_ear, left_shoulder, (left_shoulder[0], 0), neck_angle, (0, 255, 0))

        if is_calibrated:
            if (shoulder_angle < shoulder_threshold or 
                neck_angle < neck_threshold or 
                nose_to_chest_distance < nose_distance_threshold or
                head_tilt_angle < head_tilt_angle_threshold or 
                shoulder_imbalance > shoulder_imbalance_threshold):
                status = "Kötü Postür" 
                color = (0, 0, 255)  
            else:
                status = "İyi Postür"
                color = (0, 255, 0)  

            #if time.time() - last_send_time >= 2:
             #   send_posture_status(status)
              #  last_send_time = time.time()  

            cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
            cv2.putText(frame, f"Omuz Açı: {shoulder_angle:.1f}/{shoulder_threshold:.1f}", (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Boyun Açı: {neck_angle:.1f}/{neck_threshold:.1f}", (10, 90), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Burun-Göğüs Mesafesi: {nose_to_chest_distance:.1f}/{nose_distance_threshold:.1f}", (10, 120), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow('Postür Kontrolü', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\akgul\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Kalibrasyon tamamlandı.
